In [2]:
# 04/09/ 2.06PM
from gensim.models.ldamodel import LdaModel
from pprint import pprint
lda_topic_model = LdaModel.load('first_topic_model')
print(lda_topic_model)
pprint(lda_topic_model.print_topics())

LdaModel(num_terms=1047, num_topics=6, decay=0.5, chunksize=100)
[(0,
  '0.012*"oil" + 0.012*"say" + 0.010*"new" + 0.010*"price" + 0.010*"market" + '
  '0.007*"bring" + 0.007*"russian" + 0.007*"propose" + 0.007*"space" + '
  '0.007*"sale"'),
 (1,
  '0.011*"customer" + 0.009*"mail" + 0.009*"election" + 0.007*"quarter" + '
  '0.007*"share" + 0.006*"new" + 0.006*"company" + 0.006*"today" + 0.006*"get" '
  '+ 0.006*"take"'),
 (2,
  '0.009*"company" + 0.009*"new" + 0.009*"good" + 0.007*"say" + 0.007*"blue" + '
  '0.007*"week" + 0.007*"business" + 0.007*"fall" + 0.007*"go" + 0.007*"disc"'),
 (3,
  '0.016*"project" + 0.010*"economy" + 0.010*"service" + 0.010*"say" + '
  '0.007*"japanese" + 0.007*"month" + 0.007*"download" + 0.007*"retail" + '
  '0.007*"company" + 0.007*"autodesk"'),
 (4,
  '0.014*"new" + 0.014*"lynn" + 0.012*"square" + 0.009*"grant" + '
  '0.009*"record" + 0.007*"car" + 0.007*"service" + 0.007*"growth" + '
  '0.007*"give" + 0.007*"state"'),
 (5,
  '0.013*"stock" + 0.011*"comp

In [3]:
# 04/09/ 2.08PM
from pprint import pprint
lda_topic_model = LdaModel.load('first_topic_model_2')
pprint(lda_topic_model.print_topics())

[(0,
  '0.015*"project" + 0.009*"see" + 0.009*"say" + 0.007*"autodesk" + '
  '0.007*"report" + 0.007*"service" + 0.005*"oil" + 0.005*"construction" + '
  '0.005*"target" + 0.005*"accord"'),
 (1,
  '0.019*"oil" + 0.013*"market" + 0.011*"company" + 0.010*"say" + '
  '0.010*"price" + 0.010*"stock" + 0.008*"fall" + 0.008*"economy" + '
  '0.006*"new" + 0.006*"day"'),
 (2,
  '0.021*"new" + 0.011*"company" + 0.009*"year" + 0.007*"say" + 0.007*"group" '
  '+ 0.007*"dell" + 0.007*"show" + 0.007*"fall" + 0.007*"mail" + '
  '0.005*"price"'),
 (3,
  '0.012*"lynn" + 0.010*"good" + 0.010*"square" + 0.008*"make" + 0.008*"year" '
  '+ 0.008*"low" + 0.008*"new" + 0.008*"grant" + 0.006*"day" + 0.006*"news"'),
 (4,
  '0.010*"sale" + 0.008*"company" + 0.008*"customer" + 0.008*"software" + '
  '0.008*"today" + 0.008*"even" + 0.005*"go" + 0.005*"say" + 0.005*"show" + '
  '0.005*"want"'),
 (5,
  '0.007*"say" + 0.007*"alligator" + 0.007*"phone" + 0.007*"blue" + '
  '0.007*"russian" + 0.007*"space" + 0.007*"ja

In [9]:
import pickle
with open('corpus.pkl', 'rb') as f:
    BoW_corpus_1 = pickle.load(f)
with open('corpus_2.pkl', 'rb') as f:
    BoW_corpus_2 = pickle.load(f)    

In [28]:
from gensim import corpora
lda_topic_model_1 = LdaModel.load('first_topic_model')
lda_topic_model_2 = LdaModel.load('first_topic_model_2')



In [29]:
print('\nPerplexity1: ', lda_topic_model_1.log_perplexity(BoW_corpus_1))
print('\nPerplexity2: ', lda_topic_model_2.log_perplexity(BoW_corpus_2))


Perplexity1:  -7.151164316012295

Perplexity2:  -7.138301739679999


In [39]:
from gensim.models.ldamulticore import LdaMulticore
from gensim.test.utils import datapath

# argument for diff fuction
# distance ({'kullback_leibler', 'hellinger', 'jaccard', 'jensen_shannon'}) – The distance metric to calculate the difference with.
#diff(other, distance='kullback_leibler', num_words=100, n_ann_terms=10, diagonal=False, annotation=True, normed=True)

m1 = LdaMulticore.load("first_topic_model")
m2 = LdaMulticore.load("first_topic_model_2")
mdiff, annotation = m1.diff(m2, distance='kullback_leibler')
topic_diff = mdiff  # get matrix with difference for each topic pair from `m1` and `m2`
print(topic_diff)

[[0.72257896 0.66864121 0.71865033 1.         0.70131706 0.73397685]
 [0.63304473 0.71399035 0.68308276 0.83490582 0.74209205 0.76593049]
 [0.7901155  0.6267878  0.63683163 0.8663773  0.89995541 0.58544019]
 [0.71266867 0.66936438 0.81367319 0.61821756 0.87177103 0.84682891]
 [0.94956359 0.91457545 0.77481477 0.57135728 0.75263556 0.72264189]
 [0.69839391 0.78952006 0.79103098 0.66969549 0.67389582 0.65396926]]


In [46]:
mdiff2, annotation2 = m1.diff(m2, distance='jensen_shannon')
topic_diff2 = mdiff2  # get matrix with difference for each topic pair from `m1` and `m2`
print(topic_diff2) # 6x6
#print(annotation2)
print(lda_topic_model_2.get_topics()) # topic-words, 6 x 1047 


[[0.7486983  0.70599048 0.74540155 1.         0.73252411 0.79562226]
 [0.67678874 0.78560521 0.74822229 0.89043306 0.81376835 0.85400075]
 [0.83833988 0.68423781 0.68300975 0.90880633 0.97578183 0.63306285]
 [0.74784541 0.70991234 0.8247254  0.67011483 0.93303448 0.89824669]
 [0.94005517 0.94658414 0.78864315 0.612523   0.76688069 0.75993174]
 [0.74701863 0.87928843 0.87091585 0.71765998 0.73760868 0.7243553 ]]
[[0.00030982 0.00030982 0.00030981 ... 0.00030981 0.00030981 0.00030981]
 [0.00273125 0.00039021 0.00039021 ... 0.00039021 0.00039021 0.00039021]
 [0.00033312 0.00033313 0.00033312 ... 0.00033312 0.00033312 0.00033312]
 [0.00024332 0.00024332 0.00024332 ... 0.00024332 0.00024332 0.00024332]
 [0.00492563 0.00947252 0.00492563 ... 0.00037891 0.00037891 0.00037891]
 [0.00043561 0.00043561 0.0004356  ... 0.00304898 0.00304898 0.00304898]]


In [ ]:
lda_model = LdaModel.load('model')
dict = corpora.Dictionary.load('model.id2word')

In [11]:
lda_dict1 = corpora.Dictionary.load('first_topic_model.id2word')
lda_dict2 = corpora.Dictionary.load('first_topic_model_2.id2word')

In [27]:
from gensim.models.coherencemodel import CoherenceModel
topics = [['new', 'lynn','square','last','grant', 'disc']
         #,['new', 'customer','mail','last','grant', 'disc']
         ]

cm = CoherenceModel(topics=topics, 
                    #model=lda_topic_model_1, 
                    corpus=BoW_corpus_1, dictionary=lda_dict1, coherence='u_mass')
coherence = cm.get_coherence()  # get coherence value
print(coherence)

-7.189275342503036
